# Hartree-Fock: Step Size

Import libraries

In [1]:
import csv
import pynof
# import energy.py library including optimization methods
import sys
import time as t
# Define energy.py path, in our case is in myCode dir, two directories above
sys.path.insert(1, '../../myCode/')
# read files with optimization functions
from energy import calc_hf_orbrot

Create molecule and choose basis set

In [ ]:
#Molecules Definition
#Times given in Seconds, gotten on a Linux PC (12th Gen Intel(R) Core(TM) i9-12900K)
#with export OMP_NUM_THREADS=8
#Time for H2O: 135 
#Time for CO2: 1052
#Total Time: 1188

h2o = pynof.molecule("""
0 1
  O  0.0000   0.000   0.116
  H  0.0000   0.749  -0.453
  H  0.0000  -0.749  -0.453
""")

co2 = pynof.molecule("""
0 1
C    0.0000    0.0000    0.0000   
O    1.1000    0.0000    0.0000   
O    -1.1000   -0.0000    0.0000 
""")

molecules = {
    "CO2": co2,
    "H2O": h2o,
}

Minimize Hartree-Fock for each algorithm and each alpha value (step size) and store data in csv file.

In [ ]:
times = {}
# Algorithms to be tested
algorithms = [ 'SD', 'CG', 'RMSPROP', 'ADAGRAD', 'ADAM' ]
# earning rate values
alpha_values = [0.002, 0.005, 0.02, 0.04, 0.08, 0.16]

molecule_list = ["H2O","CO2"]
#Basis set definition

basis = "cc-pvdz"
#Total start time
t_st = t.time()

for molecule in molecule_list:
    #Start time for molecule
    st = t.time()
 
    #Define system
    mol = molecules[molecule]
    p = pynof.param(mol,basis)
    p.maxit = 60

    for alg in algorithms:
        p.orb_method=alg
        filename = f"{molecule}_{alg}.csv"
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["alpha", "iteration", "energy"])  # Header

            for alpha in alpha_values:
                p.alpha = alpha
                #Run HF calc for each case
                _, energy_data = calc_hf_orbrot(mol, p)
                for i_ext, energy in energy_data:
                    writer.writerow([alpha, i_ext, energy])
    #End time for molecule
    et = t.time()
    times[molecule]=et-st

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /home/pobmelat/miniconda3/envs/EduPynof/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /home/pobmelat/miniconda3/envs/EduPynof/share/psi4/basis/cc-pvdz.gbs 

Number of basis functions                   (NBF)    = 24
Inactive Doubly occupied orbitals up too     (NO1)    = 0
No. considered Strongly Doubly occupied MOs (NDOC)   = 5
No. considered Strongly Singly occupied MOs (NSOC)   = 0
No. of Weakly occ. per St. Doubly occ.  MOs (NCWO)   = 3
Dimension of the Nat. Orb. subspace         (NBF5)   = 20
No. of electrons                                     = 10
Multiplicity                                         = 1

No violations of the orthonormality

HF Calculation (SD Optimization)

Nitext  Nit_orb  Nit_occ      Eelec           Etot          Ediff        Grad_orb   Grad_occ Conv Orb Conv Occ
alpha =  0.002
Starting SD ... with alp

In [4]:
#Total start  time
t_et = t.time()

#Print times per molecule
for molecule in molecule_list:
    print(f"Time for {molecule}: {times[molecule]:.4f} s")

print(f"Total Time:",t_et-t_st)

Time for H2O: 135.9986 s
Time for CO2: 1052.2785 s
Total Time: 1188.2812004089355
